##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Modellerin kaydedilmesi ve yuklenmesi

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Modelimizin gelisimini egitim sirasinda ya da sonrasinda kaydedebiliriz. Bu sayede model egitimine kaldigi yerden devam edebilir ve uzun suren egitimleri bolebiliriz. Ayrica kaydettigimiz modelimizi baskalari ile paylasarak calismamizin tekrarlanmasini saglamamiz kolaylasir. Makine ogrenmesi calisanlar, arastirma modellerini ve tekniklerini yayimlarken sunlari paylasirlar:

* modeli yeniden yaratacak kod ve
* modelin egitildigi agirliklar ve degistirgeler

Bu bilgilerin paylasilmasi modelin nasil calistiginin anlasilmasina ve yeni verilerle kullanilmasina yardimci olur.

Dikkat: guvenilmeyen kodlara dikkat edin—TensorFlow modelleri kodtur. TensorFlow'u guvenli kullanmak icin [buraya bakiniz](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md).

### Secenekler

Kullandiginiz API'ya gore TensorFlow modellerini kaydetmek icin cesitli yontemler vardir. Bu kitapcikta yuksek-seviye API olan [tf.keras](https://www.tensorflow.org/guide/keras) kullanarak TensorFlow'da modelleri olusturup egitiyoruz. Diger yontemler icin lutfen TensorFlow [Save and Restore](https://www.tensorflow.org/guide/saved_model) ya da [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving) kilavuzlarina bakabilirsiniz.

## Kurulum

### Yuklemeler ve iceri aktarimlar

TensorFlow ve bagimliliklari yukleyelim ve iceri aktaralim:

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Modelleri HDF5 biciminde kaydetmek icin gereklidir

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Ornek veri seti alalim

[MNIST veri setini](http://yann.lecun.com/exdb/mnist/) kullanarak agirliklari nasil kaydedebilecegimizi ve yukleyecegimizi gorelim. Islemi biraz hizlandirmak icin ilk 1000 ornegi kullanalim:

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Modeli tanimlayalim

Basit bir sequential (sirali) model olusturarak baslayalim:

In [0]:
# Basit sirali modelimizi tanimlayayim
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Basit modelimizi olusturalim
model = create_model()

# Modelimizin yapisini gosterelim 
model.summary()

## Egitim sirasinda denetim noktalarin kaydedelim

Egitilmis bir modeli tekrar tekrar kullanabiliriz ya da egitim herhangibir sebebten yarim kaldiysa kaldigimiz yerden devam edebiliriz. Modelimizi egitim *sirasinda* ve egitim *sonunda* `tf.keras.callbacks.ModelCheckpoint` geri cagrimasini kullanarak surekli kaydedebiliriz.

### Denetim noktasi geri cagirmasinin kullanimi

Sadece egitim sirasinda agirliklari kaydeden bir `tf.keras.callbacks.ModelCheckpoint` geri cagirmasini olusturalim:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Modelin agirliklarini kaydeden bir geri cagirma olusturalim
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Modelimizi yeni geri cagirma ile egitelim
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# Bu asamada en iyilestiricinin durumunun kaydedilmesi ile ilgili bir uyari gorebilirsiniz.
# Bu ve bunun gibi uyarilar geri kalmis kullanimlari onlemek icin konulmustur ve goz ardi edilebilir.

Her devrin sonunda guncellenen TensorFlow denetim noktalari dosyalarini bir yerde toplayalim:

In [0]:
!ls {checkpoint_dir}

Yeni egitilmemis bir model olusturalim. Bir modeli sadece varolan agirliklardan yeniden olusturmak icin agirliklarin alindigi model ile ayni yapida bir model kullanmamiz gerekir. Ayni model yapisini kullandigimiz icin modelin farkli *olusumu* olmasina ragmen agirliklari rahatlikla kullanabiliriz.

Simdi yeni, egitilmemis bir modeli yeniden olusturalim ve test setinde degerlendirelim. Egitilmemis bir model sansa dayali sonuclar (yaklasik %10 dogrulukla) verecektir:

In [0]:
# Basir bir model olusturalim
model = create_model()

# Modeli degerlendirelim
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Simdi de denetim noktalarindan aldigimiz agirliklari yukleyelim ve yeniden degerlendirelim:

In [0]:
# Agirliklari yukleyelim
model.load_weights(checkpoint_path)

# Modeli yeniden degerlendirelim
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Denetim noktari geri cagirma secenekleri

Geri cagirma yontemi denetim noktalari icin ozgun isimler verme ve denetim noktasi yogunlugunu ayarlama gibi bircok secenek onerir.

Yeni modelimizi egitelim ve ozgunce isimlendirilmis denetim noktalarimizi her bes devirde bir kaydedelim:

In [0]:
# Dosya ismine devir sayisini ekleyelim (`str.format` kullanilir)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Her 5 devirde bir modelin agirliklarini kaydeden bir geri cagirma olusturalim
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Yeni bir model olusturalim
model = create_model()

# `checkpoint_path` bicimini kullanarak agirliklari kaydedelim
model.save_weights(checkpoint_path.format(epoch=0))

# Modeli yeni geri cagirma ile egitelim
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

Simdi elde edilen denetim noktalarina bakalim ve en sonuncusunu secelim:

In [0]:
! ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Not: varsayilan TensorFlow bicimi sadece son 5 denetim noktasini kaydeder.

Test etmek icin modeli sifirlayalim ve sonuncu denetim noktasini yukleyelim:

In [0]:
# Yeni bir model olusturalim
model = create_model()

# Daha onceden kaydedilen agirliklari yukleyelim
model.load_weights(latest)

# Modeli yeniden degerlendirelime
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Bu dosyalar neler?

Yukaridaki kod ile agirliklari [denetim noktalari](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables) toplamina kaydediyoruz-bu dosyalar sadece egitim agirliklarini ikili bicimde saklar. Denetim noktalarinda sunlar bulunur:
* Modelinizin agirliklarini bulunduran bir ya da daha fazla parca.
* Hangi agirligin hangi parcada kaydedildigini belirten bir dizin dosyasi.

Eger modelinizi sadece bir makinede egitiyorsaniz, `.data-00000-of-00001` son ekine sahip bir parcaniz olacaktir.

## Agirliklari el ile kaydedelim

Agirliklari modelimize nasil yukleyebilecegimizi gorduk. El ile kaydetmek de `Model.save_weights` yontemi ile yuklemek kadar kolay. Varsayilan ayarlarda, `tf.keras`-ve ozellikle `save_weights`— TensorFlow [denetim noktalari](../../guide/keras/checkpoints) bicimini `.ckpt` uzatmasi ile kaydeder ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html) biciminde `.h5` uzatmasi ile kaydetmeyi [Modelleri kaydedin ve dizisellestirin](../../guide/keras/saving_and_serializing#weights-only_saving_in_savedmodel_format) kilavuzunda bulabilirsiniz):

In [0]:
# Agirliklari kaydedelim
model.save_weights('./checkpoints/my_checkpoint')

# Yeni modelimizi olusturalim
model = create_model()

# Agirliklari yeniden yukleyelim
model.load_weights('./checkpoints/my_checkpoint')

# Modeli degerlendirelim
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Butun modelin kaydedilmesi

Modeller ve en iyilestiricileri bulunduklari durumu (agirliklar ve degiskenler) ve modelin ayarlarinin bulundugun bir dosyaya kaydedilebilir. Bu sayede modelinizi orjinal Python kodu olmadan da disari aktarabilirsiniz. Ayrica en iyilestirici durumu kurtarilabilir oldugu icin modelin egitimine kaldiginiz yerden devam edebilirsiniz.

Tamamen calisan bir modeli kaydetmek cok kullanislidir-TensorFlow.js'e yukleyip modellerinizi internet tarayicilarinda egitip calistirabilirsiniz ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)), ya da TensorFlow Lite ile akilli telefonlarinizda kullanabilirsiniz ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

### Modelinizin HDF5 dosyasi olarak kaydedilmesi

Keras modelinizi [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standardinda kaydetmenizi de saglar. Burdaki amacimiz icin kaydedilen modelimize tek bir ikili kutle olarak davranabiliriz:

In [0]:
# Yeni bir model olusturalim
model = create_model()

# Modeli egitelim
model.fit(train_images, train_labels, epochs=5)

# Butun modeli HDF5 dosyasina kaydedelim
model.save('my_model.h5')

Simdi modeli bu dosyadan yeniden yaratalim:

In [0]:
# Ayni modeli agirliklari ve en iyilestiricisi ile yeniden yaratalim
new_model = keras.models.load_model('my_model.h5')

# Modelin yapisini gosterelim
new_model.summary()

Dogrulugunu kontrol edelim:

In [0]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Bu yontemle her seyi kaydedebiliriz:

* Agirlik degerlerini
* Modelin ayarlarini (yapisini)
* En iyilestiricinin ayarlarini

Keras modelin yapisini inceleyerek kaydeder. Su anda TensorFlow en iyilestiricilerini (`tf.train`) kaydetme kapasitesine sahip degildir. Bunlari kullanirken modeli yukledikten sonra yeniden derlemeniz gerekir ve en iyilestiricinin durumunu kaybedersiniz.


### `saved_model` olarak

Dikkat:`tf.keras` modelin bu sekilde kaydedilmesi deneyseldir ve gelecekte degisebilir.

Modeli olusturun ve egitin:

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

`saved_model` olusturalim ve `tf.keras.experimental.export_saved_model` kullanarak zaman damgali bir dizine yerlestirelim:

In [0]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Kaydedilen modelleri listeleyelim:

In [0]:
!ls saved_models/

Kaydedilen modellerden yeni bir Keras modeli yukleyelim:

In [0]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)

# Modelin yapisina goz atalim
new_model.summary()

Yeniden yuklenen model ile bir tahmin yurutelim:

In [0]:
model.predict(test_images).shape

In [0]:
# Model degerlendirme oncesinde derlenmelidir.
# Eger modeliniz sadece dagitiliyorsa bu isleme gerek yoktur.

new_model.compile(optimizer=model.optimizer,  # Keep the optimizer that was loaded
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Yeniden yuklenen modeli degerlendirelim
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))